# 서울특별시 방배동의 요기요 식당 정보
- 매장명, 평점평균, 리뷰수, 사장님답글수, 최소주문금액 가져오기 

In [1]:
import requests
import pandas as pd
import numpy as np
import os

In [2]:
url = 'https://www.yogiyo.co.kr/api/v1/restaurants-geo/?items=60&lat=37.4835309976812&lng=126.991938810326&order=rank&page=0&search='
response = requests.get(url)
print(response)  # 400 에러 Bad Request
print(response.text) # 에러의 내용

<Response [400]>
[["error", "API key or secret is missing"]]


In [3]:
# 400 에러 가 뜬다
# 접근이 막혀있다.
# 에러메세지는 API key or secret is missing 

웹크롤러 같은 봇이 함부로 자사의 사이트 정보를 크롤링 하지 못하게 필터링 하는 다양한 방법이 있는데  
그중에 하나가 request 시 추가 되는 header 정보를 사용하는 방법이다
1. header 정보를 통해 정상적인 request 인지 확인하거나
2. header 에 특수한 정보를 넣어서 그 정보가 있는 경우에만 정상적으로 response 한다든가

In [4]:
# API key 와 secret을 찾아서 header 에 추가해보자
# 근데 API key를 어떻게 찾을까????
# 우리에겐 구글 크롬이 있다


In [5]:
# 그 전에 파이썬으로 request 를 할때 헤더는 어떻게 생겼는지 한번 보도록 하자

py_header = requests.get('https://httpbin.org/headers')  # header를 볼수 있는 url
print(py_header.text)

{
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.25.1", 
    "X-Amzn-Trace-Id": "Root=1-618ba606-5807c8ef4ae3c2fa4715e9c4"
  }
}



In [6]:
# 헤더에 인위적으로 api 키를 추가해서 요기요 서버를 한번 속여보자

In [7]:
#headers 정보 추가
headers = {
  "X-ApiKey": "iphoneap",  # API key
  "X-ApiSecret": "fe5183cc3dea12bd0ce299cf110a75a2",  # secret
  'User-agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36"  # 크롬에서 리퀘스트 한척 하기
}

In [8]:
response = requests.get(url, headers=headers) # 헤더 정보 수정&추가
print(response)

# ↓↓↓↓ 200 떳따!

<Response [200]>


In [9]:
# json() 함수로 json 으로 변환하여 새로운 변수에 담는다.
ygy_data =  response.json()
ygy_data

{'pagination': {'per_page': 60,
  'total_objects': 2371,
  'current_page': 0,
  'total_pages': 40},
 'restaurants': [{'phone_order': True,
   'free_delivery_threshold': 0,
   'section_pos': 1,
   'adjusted_delivery_fee': 2000,
   'is_available_delivery': True,
   'new_rating': 0.2523366,
   'app_order': True,
   'lng': 126.9860396,
   'list_pos': 1,
   'open': True,
   'id': 54915,
   'estimated_delivery_time': '25~35분',
   'review_count': 1375,
   'discount_from': None,
   'is_deliverable': True,
   'section': 'superlist',
   'restaurant_type': 'food',
   'franchise_id': 82,
   'thumbnail_url': 'https://rev-static.yogiyo.co.kr/franchise/thumbnail/20181004182813010545_d61c21551fb145915b554bc893ffe4f5_tn.jpg',
   'reachable': True,
   'additional_discount': 0,
   'new': False,
   'franchise_name': '주식회사김가네 / 김가네',
   'thumbnail_message': '',
   'begin': '09:30:00',
   'is_available_pickup': False,
   'tags': [],
   'representative_menus': '',
   'lat': 37.49387,
   'end': '20:20:00',
  

In [10]:
ygy_data['restaurants'] # 식당정보가 담긴 dict

[{'phone_order': True,
  'free_delivery_threshold': 0,
  'section_pos': 1,
  'adjusted_delivery_fee': 2000,
  'is_available_delivery': True,
  'new_rating': 0.2523366,
  'app_order': True,
  'lng': 126.9860396,
  'list_pos': 1,
  'open': True,
  'id': 54915,
  'estimated_delivery_time': '25~35분',
  'review_count': 1375,
  'discount_from': None,
  'is_deliverable': True,
  'section': 'superlist',
  'restaurant_type': 'food',
  'franchise_id': 82,
  'thumbnail_url': 'https://rev-static.yogiyo.co.kr/franchise/thumbnail/20181004182813010545_d61c21551fb145915b554bc893ffe4f5_tn.jpg',
  'reachable': True,
  'additional_discount': 0,
  'new': False,
  'franchise_name': '주식회사김가네 / 김가네',
  'thumbnail_message': '',
  'begin': '09:30:00',
  'is_available_pickup': False,
  'tags': [],
  'representative_menus': '',
  'lat': 37.49387,
  'end': '20:20:00',
  'discount_until': None,
  'categories': ['분식', '한식', '프랜차이즈'],
  'review_avg': 4.6,
  'min_order_amount': 15000,
  'distance': 1.26198076014,
  '

In [48]:
# list comprehension

# 담기로 한 정보 : 매장명, 평점평균, 리뷰수, 사장님답글수, 할인정보, 배달료정보, 최소주문금액

result = [
    {
        "매장명":el.get('name').strip(),
        "평점평균":el.get('review_avg'),
        "리뷰수":el.get('review_count'),
        "사장님답글수":el.get('owner_reply_count'),
        "최소주문금액":el.get('min_order_amount'),
    }
    for el in ygy_data['restaurants']
]

result

[{'매장명': '김가네-방배본동점', '평점평균': 4.6, '리뷰수': 1375, '사장님답글수': 51, '최소주문금액': 15000},
 {'매장명': '텍사스바베큐-사당점', '평점평균': 4.9, '리뷰수': 18, '사장님답글수': 4, '최소주문금액': 20000},
 {'매장명': '청년피자-사당방배점',
  '평점평균': 4.9,
  '리뷰수': 1582,
  '사장님답글수': 706,
  '최소주문금액': 15900},
 {'매장명': '후라이드참잘하는집-사당점',
  '평점평균': 4.7,
  '리뷰수': 4082,
  '사장님답글수': 470,
  '최소주문금액': 14000},
 {'매장명': '두마리찜닭두찜-서초방배점',
  '평점평균': 4.9,
  '리뷰수': 886,
  '사장님답글수': 768,
  '최소주문금액': 16000},
 {'매장명': '코리엔탈깻잎두마리치킨-낙성대점',
  '평점평균': 4.9,
  '리뷰수': 1082,
  '사장님답글수': 945,
  '최소주문금액': 12000},
 {'매장명': '놀부보쌈족발-서초점',
  '평점평균': 4.6,
  '리뷰수': 711,
  '사장님답글수': 693,
  '최소주문금액': 12000},
 {'매장명': '미스터보쌈족발-낙성대점',
  '평점평균': 4.8,
  '리뷰수': 1410,
  '사장님답글수': 14,
  '최소주문금액': 18900},
 {'매장명': '7번가피자-방배사당점',
  '평점평균': 4.9,
  '리뷰수': 986,
  '사장님답글수': 942,
  '최소주문금액': 15000},
 {'매장명': '컬투치킨-서초1호점',
  '평점평균': 4.8,
  '리뷰수': 1171,
  '사장님답글수': 1020,
  '최소주문금액': 16000},
 {'매장명': '쫄면주는삼겹본능by놀부-방배점',
  '평점평균': 4.8,
  '리뷰수': 968,
  '사장님답글수': 959,
  '최소주문금액': 17000},
 {'매장명': '또래오래-

In [49]:
df = pd.DataFrame(result)
df.head(10)  # 인덱스 순서 상위10개 

,매장명,평점평균,리뷰수,사장님답글수,최소주문금액
0,김가네-방배본동점,4.6,1375,51,15000
1,텍사스바베큐-사당점,4.9,18,4,20000
2,청년피자-사당방배점,4.9,1582,706,15900
3,후라이드참잘하는집-사당점,4.7,4082,470,14000
4,두마리찜닭두찜-서초방배점,4.9,886,768,16000
5,코리엔탈깻잎두마리치킨-낙성대점,4.9,1082,945,12000
6,놀부보쌈족발-서초점,4.6,711,693,12000
7,미스터보쌈족발-낙성대점,4.8,1410,14,18900
8,7번가피자-방배사당점,4.9,986,942,15000
9,컬투치킨-서초1호점,4.8,1171,1020,16000


In [55]:
# 리뷰수 기준 상위 20개 가게
df.sort_values('리뷰수', ascending=False)[:20]

,매장명,평점평균,리뷰수,사장님답글수,최소주문금액
51,반올림피자샵-사당방배점,4.9,4287,1331,15900
3,후라이드참잘하는집-사당점,4.7,4082,470,14000
53,황궁쟁반짜장-서경,4.4,2400,3,9000
46,동원-방배점,4.5,1895,1647,10000
2,청년피자-사당방배점,4.9,1582,706,15900
56,청년피자-사당방배점,4.9,1582,706,15900
48,만다린-방배점,4.6,1468,0,11000
7,미스터보쌈족발-낙성대점,4.8,1410,14,18900
0,김가네-방배본동점,4.6,1375,51,15000
50,맛카오치킨,4.8,1361,1106,10800


In [17]:
# 평점평균 기준 상위 20개 가게
df.sort_values('평점평균', ascending=False)[:20]

,매장명,평점평균,리뷰수,사장님답글수,최소주문금액
31,족보잇는국밥&밀면-방배점,5.0,74,0,15000
1,텍사스바베큐-사당점,4.9,18,4,20000
2,청년피자-사당방배점,4.9,1582,706,15900
4,두마리찜닭두찜-서초방배점,4.9,886,768,16000
5,코리엔탈깻잎두마리치킨-낙성대점,4.9,1082,945,12000
56,청년피자-사당방배점,4.9,1582,706,15900
23,흥부찜닭by놀부-방배점,4.9,495,493,15000
8,7번가피자-방배사당점,4.9,986,942,15000
55,7번가피자-방배사당점,4.9,986,942,15000
51,반올림피자샵-사당방배점,4.9,4287,1331,15900


In [ ]:
# 근데 평점 평균으로는 리뷰수가 적은 가게도 높게 나오기 때문에 신뢰할만한 표본으로 보기 어렵다
# 뭔가 맛집 반영이 제대로 되지 않는거 같다.
# 최소 리뷰수 300이 넘는 가게만 모아서 따로 순위를 내보자

In [57]:
review_count_over300 = df[df['리뷰수']>= 300]
review_count_over300[:20]

,매장명,평점평균,리뷰수,사장님답글수,최소주문금액
0,김가네-방배본동점,4.6,1375,51,15000
2,청년피자-사당방배점,4.9,1582,706,15900
3,후라이드참잘하는집-사당점,4.7,4082,470,14000
4,두마리찜닭두찜-서초방배점,4.9,886,768,16000
5,코리엔탈깻잎두마리치킨-낙성대점,4.9,1082,945,12000
6,놀부보쌈족발-서초점,4.6,711,693,12000
7,미스터보쌈족발-낙성대점,4.8,1410,14,18900
8,7번가피자-방배사당점,4.9,986,942,15000
9,컬투치킨-서초1호점,4.8,1171,1020,16000
10,쫄면주는삼겹본능by놀부-방배점,4.8,968,959,17000


In [58]:
review_count_over300 = review_count_over300.sort_values('평점평균', ascending=False).reset_index(drop=True)
review_count_over300.index += 1
review_count_over300[:20]

,매장명,평점평균,리뷰수,사장님답글수,최소주문금액
1,두마리찜닭두찜-서초방배점,4.9,886,768,16000
2,코리엔탈깻잎두마리치킨-낙성대점,4.9,1082,945,12000
3,청년피자-사당방배점,4.9,1582,706,15900
4,흥부찜닭by놀부-방배점,4.9,495,493,15000
5,7번가피자-방배사당점,4.9,986,942,15000
6,청년피자-사당방배점,4.9,1582,706,15900
7,7번가피자-방배사당점,4.9,986,942,15000
8,구백냥곱창-이수직영점,4.9,543,6,10500
9,반올림피자샵-사당방배점,4.9,4287,1331,15900
10,컬투치킨-서초1호점,4.8,1171,1020,16000


In [54]:
# 리뷰수 최소 300 이상인 가게 중 평점 높은 가게 top 10
result1 = review_count_over300[:10]
result1

,매장명,평점평균,리뷰수,사장님답글수,최소주문금액
1,두마리찜닭두찜-서초방배점,4.9,886,768,16000
2,코리엔탈깻잎두마리치킨-낙성대점,4.9,1082,945,12000
3,청년피자-사당방배점,4.9,1582,706,15900
4,흥부찜닭by놀부-방배점,4.9,495,493,15000
5,7번가피자-방배사당점,4.9,986,942,15000
6,청년피자-사당방배점,4.9,1582,706,15900
7,7번가피자-방배사당점,4.9,986,942,15000
8,구백냥곱창-이수직영점,4.9,543,6,10500
9,반올림피자샵-사당방배점,4.9,4287,1331,15900
10,컬투치킨-서초1호점,4.8,1171,1020,16000
